In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import time
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# NLTK tools for text processing
import re, nltk, unicodedata, string
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity =  'all'
nltk.download('stopwords')
nltk.download('punkt')

from deep_translator import GoogleTranslator
from textblob import TextBlob

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Mag\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Mag\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
review_df = pd.read_csv('../data/olist_order_reviews_dataset.csv')

In [5]:
review_df.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


In [7]:
def preprocess_comments(text):
    comments = []
    stop_words = set(stopwords.words('portuguese'))

    for words in text:
        only_letters = re.sub(r'[^a-zA-ZÀ-ÿ]', " ", words)
        lower_case = only_letters.lower()
        filtered_result = ' '.join([l for l in lower_case.split() if l not in stop_words]) 
        comments.append(filtered_result)
    #     comments = [i.strip() for i in comments]
    # for i in range(len(comments)):
    #     comments[i] = comments[i].translate(str.maketrans('','', string.punctuation))
    return comments     

In [19]:
def tokenize_comments(comment):
    tokenized_comments = []
    # for comment in comments:
    tokens = word_tokenize(comment, language= 'portuguese')
    clean_tokens = preprocess_comments(tokens)
    while("" in clean_tokens):
        clean_tokens.remove("")
    clean_tokens = ' '.join([str(elem) for elem in clean_tokens])
    tokenized_comments.append(clean_tokens)
    return tokenized_comments

In [9]:
def translate(text):
    translated = GoogleTranslator(source='auto', target='en').translate(text)
    return translated

In [10]:
def getPolarity(text):
    analysis = TextBlob(text)
    if not pd.isna(text): 
        if text == 'boa' or text =='bom' or text =='recomendo':
            result = 0.4
            return result
        elif text != '':    
            result = analysis.sentiment.polarity
            return result           
    else: return 100

In [ ]:
# def getSubjectivity(text):
#   analysis = TextBlob(text)
#   if not pd.isna(text):
#     if text != '':    
#       result = analysis.sentiment.subjectivity
#       return result
#   else: return 100

In [11]:
def getSentimentClass(polarity):    
    if polarity >= 0.3 and polarity <=1:
        return 'positive'
    elif polarity >= 0 and polarity < 0.3:
        return 'neutral'
    elif polarity < 0:
        return 'negative'        
    else: return 0

In [12]:
def reviewScore (score):
    if score >= 4:
        result = 'positive'
        return result
    elif score >= 3:
        result = 'neutral'
        return result
    elif score <= 2:
        result = 'negative'
        return result

In [17]:
def sentimentAnalysis (row):
    if row.review_comment_message != np.NaN:
        if row.review_comment_message != '':
            token = tokenize_comments(row.review_comment_message)
            trans = translate(token)
            polar = getPolarity(trans)
            clas = getSentimentClass(polar)
    elif row.review_comment_title != np.NaN:
        if row.review_comment_message != '':
            token = tokenize_comments(row.review_comment_message)
            trans = translate(token)
            polar = getPolarity(trans)
            clas = getSentimentClass(polar)
    elif row.review_score != np.NaN:
        clas = reviewScore(row.review_score)
    else: return 'neutral'
    return clas

In [25]:
review_df.apply(sentimentAnalysis, axis = 1)

TypeError: expected string or bytes-like object